# Healthcare Python Streaming Application Demo
This application demonstrates how users can develop Python Streaming Applications from a Jupyter Notebook. The Jupyter Notebook ultimately submits two Streams applications to a local Streams cluster. The first application is a pre-compiled SPL application that simulates patient waveform and vital data, as well as publishes the data to a topic. The second application is a Python Topology application written using the Streams Python API. This application subscribes to the topic containing the patient data, performs analysis on the waveforms and sends all of the data, including the results of the analysis, to the Streams view server.

Submitting the Python application from the Notebook allows for connecting to the Streams view server in order to retrieve the data. Once the data has been retrieved, it can be analyzed, manipulated or visualized like any other data accessed from a notebook. In the case of this demo, waveform graphs and numerical widgets are being used to display the healthcare data of the patient. 

The following diagram outlines the architecture of the demo.  

![Demo Architecture](../images/architecture_diagram.jpg)

### Cell Description

This cell is responsible for building and submitting the Streams applications to the Streams cluster. 

###### PhysionetIngestServiceMulti microservice

This microservice comes in the form of a pre-compiled SAB file. The microservice retrieves patient waveform and vital data from a Physionet database (https://www.physionet.org/). The patient data is submitted to the *ingest-physionet* topic, which allows it to be consumed from downtstream applications or services. 

##### Healthcare Patient Python Topology Application

The main source code for the Python Topology application can be found in `src/healthcare_patient.py`. As described in the above architecture, this is a Streaming Python application that ingests the patient data from the *ingest-physionet* topic, performs filtering and analysis on the data, and then sends the data to the Streams view server. 

For reference, the following is a snippet of the underlying Python Topology application code: 

    class HealthcarePatientData:
        def __init__(self, username, password, sample_rate=125, patient_id=None):
            self.username = username
            self.password = password
            self.sample_rate = sample_rate
            self.target_sample_rate = 100 
            self.patient_id=patient_id

        def run(self, context='DISTRIBUTED'):
            ## Create topology
            topo = Topology('HealthcareDemo')

            ## Ingest, preprocess and aggregate patient data
            patientData = topo.subscribe('ingest-physionet', schema.CommonSchema.Json) \
                              .map(functions.identity) \
                              .filter(healthcare_functions.PatientFilter(self.patient_id)) \
                              .transform(healthcare_functions.GenTimestamp(self.sample_rate)) \
                              .transform(SlidingWindow(length=self.sample_rate, trigger=self.sample_rate-1)) \
                              .transform(healthcare_functions.aggregate) \

            ## Calculate RPeak and RR delta
            rpeak_data_stream = patientmonitoring_functions.streaming_rpeak(patientData, self.sample_rate, data_label='ECG Lead II')

            ## Create a view of the data
            self.view_data = rpeak_data_stream.view()

            ## Compile Python Streams application and submit job
            streamsx.topology.context.submit(context, topo.graph)


        '''Access view data'''
        def get_data(self):
            return self.view_data.start_data_fetch()



In [ ]:
import sys,os,os.path
sys.path.append('../src')
sys.path.append('../ext/biosppy_streaming')
from healthcare_patient import HealthcarePatientData
from healthcare_patient import PatientIngestService
import subprocess
import getpass
import json
from streamtool import Streamtool as st

vcap_ = ''  # streaming analytics service VCAP
user_ = ''  # username for distributed instance
pass_ = ''  # password for distributed instance

# supply context for submitting build
context = 'DISTRIBUTED'
# context = 'ANALYTICS_SERVICE'

if context == 'DISTRIBUTED':
    print ('Submitting to a distributed instance.')

    user_ = input('Username: ')
    pass_ = getpass.getpass(prompt='Password: ')

    ## display Streams Console link
    print("Streams Console: ", end='')
    st.geturl()
    
    ## submit patient ingest microservice
    PatientIngestService(num_patients=1).run()

elif context == 'ANALYTICS_SERVICE':
    print ('Submitting to streaming analytic service.')
  
    # Assume you have created this file by running the "Create VCAP Service Credential" notebook
    with open('vcap_services.json') as json_data:
        vcap_ = json.load(json_data)
        
    # Need to submit patient ingest microservice manually

else:
    print ('Unknown context "%s" specified.' % (context))
    assert (False)
    
print('')

## submit patient analysis job (Python Topology Application)
healthcare_patient = HealthcarePatientData(vcap=vcap_, username=user_, password=pass_, patient_id="patient-1")
healthcare_patient.run(context=context)

print('DONE')

### Cell Description

This cell initializes all of the graphs that will be used as well as creates the background job that access the view data.

The view data is continuously retrieved from the Streams view server in a background job. Each graph object receives a copy of the data. The graph objects extracts and stores the data that is relevant for that particular graph. Each time a call to ```update()``` is made on a graph object, the next data point is retrieved and displayed. Each graph object maintains an internal queue so that each time a call to ```update()``` is made, the next element in the queue is retrieved and removed. 

In [ ]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.layouts import column, row, gridplot, widgetbox
from bokeh.models import Range1d, BasicTicker
from bokeh.io import output_notebook, push_notebook
import medgraphs
from utils import *
import time
import collections

## load BokehJS visualization library (must be loaded in a separate cell)
from bokeh.io import output_notebook, push_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
%autosave 0
%reload_ext autoreload
%aimport utils
%aimport medgraphs
%autoreload 1


## create the graphs ##
graphs = []

ecg_leadII_graph = medgraphs.ECGGraph(signal_label='ECG Lead II', title='ECG Lead II', plot_width=600, min_range=-0.5, max_range=2.0)
graphs.append(ecg_leadII_graph)

leadII_poincare = medgraphs.PoincareGraph(signal_label='Poincare - ECG Lead II', title='Poincare - ECG Lead II')
graphs.append(leadII_poincare)

ecg_leadV_graph = medgraphs.ECGGraph(signal_label='ECG Lead V', title='ECG Lead V', plot_width=600)
graphs.append(ecg_leadV_graph)

resp_graph = medgraphs.ECGGraph(signal_label='Resp', title='Resp', min_range=-1, max_range=3, plot_width=600)
graphs.append(resp_graph)

pleth_graph = medgraphs.ECGGraph(signal_label='Pleth', title='Pleth', min_range=0, max_range=5, plot_width=600)
graphs.append(pleth_graph)

hr_numeric = medgraphs.NumericText(signal_label='HR', title='HR', color='#7cc7ff')
graphs.append(hr_numeric)

pulse_numeric = medgraphs.NumericText(signal_label='PULSE', title='PULSE', color='#e71d32')
graphs.append(pulse_numeric)

spo2_numeric = medgraphs.NumericText(signal_label='SpO2', title='SpO2', color='#8cd211')
graphs.append(spo2_numeric)

abp_numeric = medgraphs.ABPNumericText(abp_sys_label='ABP Systolic', abp_dia_label='ABP Diastolic', title='ABP', color='#fdd600')
graphs.append(abp_numeric)

## retrieve data from Streams view in a background job ##
def data_collector(view, graphs):
    for d in iter(view.get, None):
        for g in graphs:
            g.add(d)
            
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()
jobs.new(data_collector, healthcare_patient.get_data(), graphs)

### Cell Description

This cell is responsible for laying out and displaying the graphs. There is an infinite loop that continuously calls the ```update()``` method on each of the graphs. After each graph has been updated, a call to ```push_notebook()``` is made, which causes the notebook to update the graphics. 

In [ ]:
## display graphs
show(
    row(
        column(
            ecg_leadII_graph.get_figure(), 
            ecg_leadV_graph.get_figure(), 
            resp_graph.get_figure(),
            pleth_graph.get_figure()
        ), 
        column(
            leadII_poincare.get_figure(),
            widgetbox(hr_numeric.get_figure()),
            widgetbox(pulse_numeric.get_figure()),
            widgetbox(spo2_numeric.get_figure()),
            widgetbox(abp_numeric.get_figure())
        )
    ),
    notebook_handle=True
)

cnt = 0
while True:
    ## update graphs
    for g in graphs:
        g.update()

    ## update notebook 
    cnt += 1
    if cnt % 5 == 0:
        push_notebook() ## refresh the graphs
        cnt = 0
    time.sleep(0.008)
